### Installation

In [40]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
    # ==0.7.3

In [41]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.7.3
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

# **Model**

Load up `DeepSeek-R1-Distill-Qwen1.5B`, and set parameters

In [42]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 8192 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,# Reduce if out of memory
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/deepseek-r1-distill-qwen-1.5b-unsloth-bnb-4bit
==((====))==  Unsloth 2025.5.4: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/deepseek-r1-distill-qwen-1.5b-bnb-4bit with actual GPU utilization = 25.84%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8192. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.04 GB. Also swap space = 0 GB.
WARNING 05-16 12:57:17 con

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-16 12:57:22 model_runner.py:1115] Loading model weights took 1.5052 GB
INFO 05-16 12:57:25 worker.py:267] Memory profiling takes 2.39 seconds
INFO 05-16 12:57:25 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.26) = 3.81GiB
INFO 05-16 12:57:25 worker.py:267] model weights take 1.51GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.89GiB; the rest of the memory reserved for KV Cache is 1.42GiB.
INFO 05-16 12:57:26 executor_base.py:111] # cuda blocks: 3318, # CPU blocks: 0
INFO 05-16 12:57:26 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 6.48x
INFO 05-16 12:57:26 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilizati


Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:37<00:00,  1.62s/it]

INFO 05-16 12:58:03 model_runner.py:1562] Graph capturing finished in 37 secs, took 0.41 GiB
INFO 05-16 12:58:03 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 40.72 seconds


Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'q_norm', 'pre_feedforward_layernorm', 'k_norm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'q_norm', 'pre_feedforward_layernorm', 'k_norm']


## Evaluation

In [43]:
!pip install -U datasets

In [44]:
from datasets import load_dataset, Dataset

In [45]:
try:
    aime24 = load_dataset("simplescaling/aime24_nofigures", revision="main")['train']
    print("Dataset loaded successfully with revision 'main'")
except Exception as e:
    print(f"Loading with revision 'main' failed: {e}")

Dataset loaded successfully with revision 'main'


In [46]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.0,
    max_tokens = max_seq_length,
)

In [47]:
example = aime24[0]
prompt = tokenizer.apply_chat_template([
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)
print(prompt)
output = model.fast_generate(
    [prompt],
    sampling_params = sampling_params,
)[0].outputs[0].text
print(output)

<｜begin▁of▁sentence｜><｜User｜>Every morning Aya goes for a $9$-kilometer-long walk and stops at a coffee shop afterwards. When she walks at a constant speed of $s$ kilometers per hour, the walk takes her 4 hours, including $t$ minutes spent in the coffee shop. When she walks $s+2$ kilometers per hour, the walk takes her 2 hours and 24 minutes, including $t$ minutes spent in the coffee shop. Suppose Aya walks at $s+\frac{1}{2}$ kilometers per hour. Find the number of minutes the walk takes her, including the $t$ minutes spent in the coffee shop.<｜Assistant｜><think>




Processed prompts: 100%|██████████| 1/1 [00:54<00:00, 54.48s/it, est. speed input: 2.55 toks/s, output: 58.77 toks/s]

Okay, so I have this problem where Aya walks a 9-kilometer walk. Depending on her speed, the total time she takes, including the time spent in the coffee shop, changes. I need to find out how long it takes her if she walks at a speed of s + 1/2 kilometers per hour. 

Let me try to break this down step by step. 

First, when she walks at s kilometers per hour, the total time taken is 4 hours, which includes t minutes spent in the coffee shop. So, the actual walking time is 4 hours minus t minutes. But wait, the problem says the total time is 4 hours, including t minutes. So, maybe the total time is 4 hours, and t is the time she spends in the coffee shop. So, the walking time is 4 - t minutes? Hmm, but that might not be necessary yet.

Wait, maybe I should convert everything into consistent units. The speed is in kilometers per hour, and the distance is in kilometers, so time will be in hours. But the t is given in minutes, so I need to convert t into hours as well.

Let me denote t as 

# Obtain pretrained results and extract answers

In [48]:
import re

def extract_boxed_value(text):
    # Use regular expression to find the value inside \boxed{}
    pattern = r'\\boxed{([^}]*)}'
    match = re.search(pattern, text)

    if match:
        return match.group(1)
    else:
        return None

def extract_answers(generated_outputs):
  answers = []
  correct_count = 0
  for x,y in zip(aime24, generated_outputs):
    x['prediction'] = y
    x['pred_ans'] = extract_boxed_value(y)
    answers.append(x)
    if (x['pred_ans'] == x['answer']):
      correct_count += 1
  accuracy = correct_count / len(aime24)
  return answers, accuracy

In [49]:
# Get output with default prompting
i = 0
pretrained_outputs_default = []
for example in aime24:
    print(f"processing {i}-th example")
    text = tokenizer.apply_chat_template([
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)

    output1 = model.fast_generate(
      [text],
      sampling_params = sampling_params,
      lora_request = None,
      )[0].outputs[0].text
    pretrained_outputs_default.append(output1)
    i +=1

default_prompt_answers, default_prompt_accuracy = extract_answers(pretrained_outputs_default)

processing 0-th example



Processed prompts: 100%|██████████| 1/1 [00:53<00:00, 53.58s/it, est. speed input: 2.59 toks/s, output: 59.76 toks/s]


processing 1-th example



Processed prompts: 100%|██████████| 1/1 [02:19<00:00, 139.67s/it, est. speed input: 0.80 toks/s, output: 57.86 toks/s]


processing 2-th example


...


Processed prompts: 100%|██████████| 1/1 [02:20<00:00, 140.11s/it, est. speed input: 0.54 toks/s, output: 57.94 toks/s]


processing 28-th example



Processed prompts: 100%|██████████| 1/1 [02:18<00:00, 138.63s/it, est. speed input: 1.20 toks/s, output: 57.90 toks/s]


processing 29-th example



Processed prompts: 100%|██████████| 1/1 [02:19<00:00, 139.82s/it, est. speed input: 0.77 toks/s, output: 57.83 toks/s]


In [53]:
print(default_prompt_accuracy)

0.1


# Using the Mathematical Prompt

In [54]:
SYSTEM_PROMPT = """
Solve the following math problem efficiently and clearly.
The last line of your response should be of the following
format: ’Therefore, the final answer is: $\boxed{ANSWER}$.
I hope it is correct’ (withoutquotes) where ANSWER is just
the final number or expression that solves the problem.
Think step by step before answering.\n
"""


In [56]:
example = aime24[0]
prompt = tokenizer.apply_chat_template([
      {'role': 'system', 'content': SYSTEM_PROMPT},
      {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)
print(prompt)
output = model.fast_generate(
    [prompt],
    sampling_params = sampling_params,
)[0].outputs[0].text
print(output)

<｜begin▁of▁sentence｜>
Solve the following math problem efficiently and clearly.
The last line of your response should be of the following
format: ’Therefore, the final answer is: $oxed{ANSWER}$.
I hope it is correct’ (withoutquotes) where ANSWER is just
the final number or expression that solves the problem.
Think step by step before answering.

<｜User｜>Every morning Aya goes for a $9$-kilometer-long walk and stops at a coffee shop afterwards. When she walks at a constant speed of $s$ kilometers per hour, the walk takes her 4 hours, including $t$ minutes spent in the coffee shop. When she walks $s+2$ kilometers per hour, the walk takes her 2 hours and 24 minutes, including $t$ minutes spent in the coffee shop. Suppose Aya walks at $s+\frac{1}{2}$ kilometers per hour. Find the number of minutes the walk takes her, including the $t$ minutes spent in the coffee shop.<｜Assistant｜><think>






Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts:  33%|███▎      | 1/3 [06:20<12:41, 380.89s/it, est. speed input: 0.56 toks/s, output: 20.95 toks/s]


Processed prompts:  67%|██████▋   | 2/3 [06:34<02:44, 164.79s/it, est. speed input: 1.09 toks/s, output: 40.46 toks/s]


Processed prompts: 100%|██████████| 3/3 [06:35<00:00, 131.79s/it, est. speed input: 1.62 toks/s, output: 60.55 toks/s]

Okay, so I have this problem where Aya walks a 9-kilometer walk every morning. The problem gives me two different speeds and the corresponding times she takes, including the time spent in the coffee shop. I need to find out how long it would take her if she walks at a speed of s + 1/2 kilometers per hour.

Let me try to break this down step by step. First, let's parse the information given.

When Aya walks at s km/h, the total time taken is 4 hours, which includes t minutes spent in the coffee shop. So, the total time is 4 hours, which is 4*60 = 240 minutes. Therefore, the time spent in the coffee shop is t minutes, and the walking time is 240 - t minutes.

Similarly, when she walks at s + 2 km/h, the total time is 2 hours and 24 minutes. Let me convert that to minutes: 2*60 + 24 = 120 + 24 = 144 minutes. So, the total time is 144 minutes, which means the time spent in the coffee shop is t minutes, and the walking time is 144 - t minutes.

Wait, hold on. The problem says that t is the 

In [59]:
# Get output with paper prompting
i = 0
pretrained_outputs_paper_prompt = []
for example in aime24:
    print(f"processing {i}-th example")
    text = tokenizer.apply_chat_template([
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)

    output2 = model.fast_generate(
      [text],
      sampling_params = sampling_params,
      lora_request = None,
      )[0].outputs[0].text
    pretrained_outputs_paper_prompt.append(output2)
    i +=1

paper_prompt_answers, paper_prompt_accuracy = extract_answers(pretrained_outputs_paper_prompt)


processing 0-th example





Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████| 1/1 [02:18<00:00, 138.60s/it, est. speed input: 1.54 toks/s, output: 57.57 toks/s]


processing 1-th example





Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████| 1/1 [02:18<00:00, 138.31s/it, est. speed input: 1.35 toks/s, output: 57.88 toks/s]


processing 2-th example


...

Processed prompts: 100%|██████████| 1/1 [02:19<00:00, 139.07s/it, est. speed input: 1.09 toks/s, output: 57.83 toks/s]


processing 28-th example


Processed prompts: 100%|██████████| 1/1 [02:17<00:00, 137.87s/it, est. speed input: 1.75 toks/s, output: 57.68 toks/s]


processing 29-th example


Processed prompts: 100%|██████████| 1/1 [02:19<00:00, 139.04s/it, est. speed input: 1.31 toks/s, output: 57.62 toks/s]


# Saving Results

In [ ]:
print(paper_prompt_answers[0])

In [1]:
import csv

def save_results(res_1, res_2, col_names):
  with open('output.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow(['Correct Answer', col_names[0], col_names[1]])
      for i in range(len(res_1)):
          writer.writerow([res_1[i]['answer'], res_1[i]['pred_ans'], res_2[i]['pred_ans']])

col_names = ['Default Prompt Answers', 'Paper Prompt Answers']
save_results(default_prompt_answers, paper_prompt_answers, col_names)

NameError: name 'default_prompt_answers' is not defined

In [57]:
import json

def save_reasoning(data, file_name):
  # Writing dictionary to a JSON file
  with open(file_name, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [58]:
save_reasoning(pretrained_outputs_default, 'default_prompt_reasoning.json')

In [61]:
save_reasoning(pretrained_outputs_paper_prompt, 'paper_prompt_reasoning.json')